In [1]:

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.stats import shapiro
from scipy.stats import levene
from scipy.stats import bartlett
from scipy.stats import ttest_ind
from pandas import DataFrame
from statsmodels.sandbox.stats.multicomp import MultiComparison
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 1)

In [2]:
df = DataFrame({'tensile' : [7,7,15,11,9,12,17,12,18,18,14,18,18,19,19,19,25,22,19,23,7,10,11,15,11],
               'pct': [15, 15, 15, 15, 15, 20, 20, 20, 20, 20, 25, 25, 25, 25, 25, 30, 30, 30, 30, 30, 35, 35, 35, 35, 35]})
df

,tensile,pct
0,7,15
1,7,15
2,15,15
3,11,15
4,9,15
5,12,20
6,17,20
7,12,20
8,18,20
9,18,20


In [3]:
unique = df["pct"].unique()
unique

array([15, 20, 25, 30, 35], dtype=int64)

In [4]:
model = ols('tensile ~ C(pct)', df)
fit = model.fit()
result = anova_lm(fit)
result

,df,sum_sq,mean_sq,F,PR(>F)
C(pct),4.0,475.76,118.94,14.756824,0.000009
Residual,20.0,161.20,8.06,NaN,NaN


### p-value가 0.05보다 작으므로 통계적으로 유의미한 차이가 있다.<br> 따라서 사후분석이 필요하다

## 정규성확인

In [5]:
for u in unique:
    s = shapiro(df['tensile'][df['pct']==u])
    print(s)
    print("%s 수준의 검정통계량: %0.2f, p-value: %0.2f\n" % (u, s.statistic, s.pvalue))

ShapiroResult(statistic=0.8810376524925232, pvalue=0.3140396773815155)
15 수준의 검정통계량: 0.88, p-value: 0.31

ShapiroResult(statistic=0.7538275718688965, pvalue=0.03228148818016052)
20 수준의 검정통계량: 0.75, p-value: 0.03

ShapiroResult(statistic=0.7387246489524841, pvalue=0.023324359208345413)
25 수준의 검정통계량: 0.74, p-value: 0.02

ShapiroResult(statistic=0.9020196199417114, pvalue=0.421148419380188)
30 수준의 검정통계량: 0.90, p-value: 0.42

ShapiroResult(statistic=0.941970705986023, pvalue=0.6799027323722839)
35 수준의 검정통계량: 0.94, p-value: 0.68



- `20%와 25%의 p-value값이 0.05 이하이므로 정규성을 벗어난다고 볼 수 있다.`
- `15% 30% 35% 의 p-value값은 0.05 이상이므로 정규성을 크게 벗어난다고 보기 어렵다.`

## 독립성 확인

In [6]:
levene(
    df['tensile'][df['pct'] == 15],
    df['tensile'][df['pct'] == 20],
    df['tensile'][df['pct'] == 25],
    df['tensile'][df['pct'] == 30],
    df['tensile'][df['pct'] == 35] )

LeveneResult(statistic=0.317948717948718, pvalue=0.8625858807756616)

- `다섯 집단의 모분산에 유의미한 차이를 발견하지 못했다. 등분산성 가정이 유지된다. (p-value > 0.05)`

In [7]:
comp = MultiComparison(df["tensile"], df["pct"])
result = comp.allpairtest(ttest_ind)
print(result[0])

Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.01, alphacBonf=0.005
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
    15     20 -2.7325 0.0257    0.2575  False
    15     25 -4.4301 0.0022     0.022   True
    15     30 -6.2191 0.0003    0.0025   True
    15     35 -0.5077 0.6254       1.0  False
    20     25 -1.3101 0.2265       1.0  False
    20     30 -3.4027 0.0093    0.0932  False
    20     35  2.4244 0.0416    0.4156  False
    25     30 -2.6846 0.0277    0.2773  False
    25     35  4.3007 0.0026    0.0261   True
    30     35  6.2354 0.0002    0.0025   True
---------------------------------------------


### pct에 따른 tensile 평균 차이는 유의미하였다(F(4, 20) = 14.756824, p < 0.05). 봉페로니 교정을 이용하여 사후분석을 실시한 결과, `15% 와 25%`,`15% 와 30%`,`25% 와 35%`,`30% 와 35%` 수준 에서 유의미한 평균 차이가 있었다(p < 0.05).

# 2)

In [8]:
df = DataFrame({'weight' : [60.8, 57.0, 65.0, 58.6, 61.7, 68.7, 67.7, 74.0, 66.3, 69.8, 102.6, 102.1, 100.2, 96.5, 87.9, 84.2, 83.1, 85.7, 90.3],
                'saryo' : [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4]})
df

,weight,saryo
0,60.8,1
1,57.0,1
2,65.0,1
3,58.6,1
4,61.7,1
5,68.7,2
6,67.7,2
7,74.0,2
8,66.3,2
9,69.8,2


In [9]:
unique = df["saryo"].unique()
unique

array([1, 2, 3, 4], dtype=int64)

In [10]:
model = ols('weight ~ C(saryo)', df)
fit = model.fit()
result = anova_lm(fit)
result

,df,sum_sq,mean_sq,F,PR(>F)
C(saryo),3.0,4226.347895,1408.782632,164.641523,1.061311e-11
Residual,15.0,128.350000,8.556667,NaN,NaN


### p-value가 0.05보다 작으므로 통계적으로 유의미한 차이가 있다.<br> 따라서 사후분석이 필요하다

In [11]:
for u in unique:
    s = shapiro(df['weight'][df['saryo']==u])
    print(s)
    print("%s 수준의 검정통계량: %0.2f, p-value: %0.2f\n" % (u, s.statistic, s.pvalue))

ShapiroResult(statistic=0.9792436361312866, pvalue=0.9305320382118225)
1 수준의 검정통계량: 0.98, p-value: 0.93

ShapiroResult(statistic=0.9234057664871216, pvalue=0.5521445274353027)
2 수준의 검정통계량: 0.92, p-value: 0.55

ShapiroResult(statistic=0.8848595023155212, pvalue=0.35980650782585144)
3 수준의 검정통계량: 0.88, p-value: 0.36

ShapiroResult(statistic=0.9610342383384705, pvalue=0.8151664137840271)
4 수준의 검정통계량: 0.96, p-value: 0.82



- `모든 집단의 p-value값은 0.05 이상이므로 정규성을 크게 벗어난다고 보기 어렵다.`

In [12]:
bartlett(
    df['weight'][df['saryo'] == 1],
    df['weight'][df['saryo'] == 2],
    df['weight'][df['saryo'] == 3],
    df['weight'][df['saryo'] == 4])

BartlettResult(statistic=0.0328424024300886, pvalue=0.998432540893909)

- `네 집단의 모분산에 유의미한 차이를 발견하지 못함(P > 0.05). 등분산성 가정이 유지됨`

In [13]:
hsd = pairwise_tukeyhsd(df['weight'], df['saryo'])
print(hsd.summary())

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     1      2     8.68 0.0015   3.3476 14.0124   True
     1      3    39.73  0.001  34.0741 45.3859   True
     1      4    25.62  0.001  20.2876 30.9524   True
     2      3    31.05  0.001  25.3941 36.7059   True
     2      4    16.94  0.001  11.6076 22.2724   True
     3      4   -14.11  0.001 -19.7659 -8.4541   True
-----------------------------------------------------


### 사료의 종류에 따른 weight 평균 차이는 유의미하였다(F(3, 15) = 164.641523, p < 0.05). 투키의 HSD를 이용하여 사후분석을 실시한 결과, 모든 요인들은 유의미한 평균 차이가 있었다(p < 0.05).

# 3)

In [14]:
df3 = DataFrame({'result' : [65,87,73,79,81,69,75,69,83,81,72,79,90,59,78,67,62,83,76,94,89,80,80],
               'method':[1,1,1,1,1,1,2,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4]})
df3

,result,method
0,65,1
1,87,1
2,73,1
3,79,1
4,81,1
5,69,1
6,75,2
7,69,2
8,83,2
9,81,2


In [15]:
unique = df3["method"].unique()
unique

array([1, 2, 3, 4], dtype=int64)

In [16]:
model = ols('result ~ C(method)', df3)
fit = model.fit()
result = anova_lm(fit)
result

,df,sum_sq,mean_sq,F,PR(>F)
C(method),3.0,559.369048,186.456349,2.855539,0.064413
Residual,19.0,1240.630952,65.296366,NaN,NaN


### - p-value값이 0.05이상이므로 각 요인간의 유의미한 차이가 없다.

# 4)

In [17]:
df4 = DataFrame({'score':[6,9,2,16, 7,7,12,10,16, 11,7,4,7,7],
                 'method':['B', 'B', 'B','B','D','D','D','D','D','S','S','S','S','S']})
df4

,score,method
0,6,B
1,9,B
2,2,B
3,16,B
4,7,D
5,7,D
6,12,D
7,10,D
8,16,D
9,11,S


In [18]:
unique = df4["method"].unique()
unique

array(['B', 'D', 'S'], dtype=object)

In [19]:
df4['method'] = df4['method'].astype('category').cat.rename_categories({'B': 1, 'D': 2, 'S': 3})
unique = df4['method'].unique()
unique

[1, 2, 3]
Categories (3, int64): [1, 2, 3]

In [20]:
model = ols('score ~ C(method)', df4)
fit = model.fit()
result = anova_lm(fit)
result

,df,sum_sq,mean_sq,F,PR(>F)
C(method),2.0,26.464286,13.232143,0.779403,0.48244
Residual,11.0,186.750000,16.977273,NaN,NaN


### - p-value값이 0.05이상이므로 각 요인간의 유의미한 차이가 없다.

# 5)

In [21]:
df5 = DataFrame({'모종': [10.8, 9.1, 13.5, 9.2, 11.1, 11.2, 8.2, 11.3, 5.4, 4.6, 7.4, 5, 5.8, 5.3, 3.2, 7.5],
                '효소': [0, 0, 0, 0, 1000, 1000, 1000, 1000, 5000, 5000, 5000, 5000, 10000, 10000, 10000, 10000]})
df5

,모종,효소
0,10.8,0
1,9.1,0
2,13.5,0
3,9.2,0
4,11.1,1000
5,11.2,1000
6,8.2,1000
7,11.3,1000
8,5.4,5000
9,4.6,5000


In [22]:
unique = df5["효소"].unique()
unique

array([    0,  1000,  5000, 10000], dtype=int64)

In [23]:
model = ols('모종 ~ C(효소)', df5)
fit = model.fit()
result = anova_lm(fit)
result

,df,sum_sq,mean_sq,F,PR(>F)
C(효소),3.0,101.1275,33.709167,12.085748,0.000615
Residual,12.0,33.4700,2.789167,NaN,NaN


### - p-value값이 0.05이하이므로 각 요인간의 유의미한 차이가 있다. 따라서 사후분석을 시행한다.

In [24]:
for u in unique:
    s = shapiro(df5['모종'][df5['효소']==u])
    print(s)
    print("%s 수준의 검정통계량: %0.2f, p-value: %0.2f\n" % (u, s.statistic, s.pvalue))

ShapiroResult(statistic=0.8557126522064209, pvalue=0.2452031970024109)
0 수준의 검정통계량: 0.86, p-value: 0.25

ShapiroResult(statistic=0.680984616279602, pvalue=0.006817749701440334)
1000 수준의 검정통계량: 0.68, p-value: 0.01

ShapiroResult(statistic=0.8542202115058899, pvalue=0.24012289941310883)
5000 수준의 검정통계량: 0.85, p-value: 0.24

ShapiroResult(statistic=0.9810736179351807, pvalue=0.908288300037384)
10000 수준의 검정통계량: 0.98, p-value: 0.91



### 효소함량 1000수준에서 p-value값이 0.05보다 작기때문에 정규성을 만족하지 못한다. <br>(문제에서 정규성을 만족한다고 가정했으므로 계속 진행)

In [25]:
levene(
    df5['모종'][df5['효소'] == 0],
    df5['모종'][df5['효소'] == 1000],
    df5['모종'][df5['효소'] == 5000],
    df5['모종'][df5['효소'] == 10000], )

LeveneResult(statistic=0.3102678571428573, pvalue=0.8176159985325269)

### levene의 테스트로 등분산성을 검정한 결과 p-value > 0.05로 등분산성을 만족한다.

In [26]:
hsd = pairwise_tukeyhsd(df5['모종'], df5['효소'])
print(hsd.summary())

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     0   1000     -0.2    0.9 -3.7065  3.3065  False
     0   5000    -5.05 0.0051 -8.5565 -1.5435   True
     0  10000     -5.2 0.0041 -8.7065 -1.6935   True
  1000   5000    -4.85 0.0069 -8.3565 -1.3435   True
  1000  10000     -5.0 0.0055 -8.5065 -1.4935   True
  5000  10000    -0.15    0.9 -3.6565  3.3565  False
----------------------------------------------------


### 효소의 함량에 따른 모종 성장의 평균 차이는 유의미하였다(F(3, 12) = 12.085748, p < 0.05). <br>투키의 HSD를 이용하여 사후분석을 실시한 결과, 효소 함량이 0인그룹과 1000인 그룹, 5000인그룹과 10000인 그룹을 제외한 요인들은 유의미한 평균 차이가 있었다(p < 0.05).

# 6)

In [27]:
from pandas import concat
df6 = DataFrame({'점수' : [7.65, 8.04, 8.35, 9.36, 8.68, 9.11],
                '전공' : ['화학', '화학', '생물', '생물', '통계학', '통계학'],
                '성별' : ['남','여','남','여','남','여']})
df6 = concat([df6, df6])
df6

,점수,전공,성별
0,7.65,화학,남
1,8.04,화학,여
2,8.35,생물,남
3,9.36,생물,여
4,8.68,통계학,남
5,9.11,통계학,여
0,7.65,화학,남
1,8.04,화학,여
2,8.35,생물,남
3,9.36,생물,여


In [28]:
print(df6['전공'].unique())
print(df6['성별'].unique())

['화학' '생물' '통계학']
['남' '여']


In [29]:
df6['전공'] = df6['전공'].astype('category').cat.rename_categories({'화학': 1, '생물': 2, '통계학': 3})
df6['성별'] = df6['성별'].astype('category').cat.rename_categories({'남': 1, '여': 2})
unique = (df6['전공'].unique(), df6['성별'].unique())
unique

([1, 2, 3]
 Categories (3, int64): [1, 2, 3],
 [1, 2]
 Categories (2, int64): [1, 2])

### 균형설계자료 확인

In [30]:
df6.groupby('전공').agg('count')

,점수,성별
전공,,
2,4,4
3,4,4
1,4,4


- 전공 요인으로 구분한 집단별 표본수는 모두 4로 동일

In [31]:
df6.groupby('성별').agg('count')

,점수,전공
성별,,
1,6,6
2,6,6


- 성별 요인으로 구분한 집단별 표본수는 모두 6으로 동일

In [32]:
df6.groupby(['전공', '성별']).agg('count')

점수
전공 성별    
2  1    2
   2    2
3  1    2
   2    2
1  1    2
   2    2

- 성별과 전공 요인으로 구분한 각 집단별 표본수는 모두 2로 동일
- 모든 집단별 표본수가 동일하므로, 균형설계자료 -> 이원분산분석 가능

In [33]:
model = ols('점수 ~ C(전공) * C(성별)', df6)
fit = model.fit()
anova_lm(fit)

,df,sum_sq,mean_sq,F,PR(>F)
C(전공),2.0,2.832267e+00,1.416133e+00,1.923388e+28,3.794573e-84
C(성별),1.0,1.116300e+00,1.116300e+00,1.516155e+28,1.936746e-83
C(전공):C(성별),2.0,2.408000e-01,1.204000e-01,1.635269e+27,6.174417e-81
Residual,6.0,4.417621e-28,7.362702e-29,NaN,NaN


### 결과분석
- 전공: F(2,6) = 1.923388e+28, p-value < 0.05로 유의미, 즉 전공 수준에 따라 평균에 차이가 난다고 볼 수 있다.
- 성별: F(1,6) = 1.516155e+28, p-value < 0.05로 유의미, 즉 성별에 따라 평균에 차이가 난다고 볼 수 있다.
- 전공, 성별: F(2,6) = 1.635269e+27, p-value < 0.05로 유의미, 즉 전공, 성별은 평균과 관련이 있다고 볼 수 있다.

### 결과보고

`점수`에 대하여 `전공`과 `성별`를 요인으로 하는 이원분산분석을 실시한 결과, `전공`의 주효과는 유의미하였으며(F(2,6) = 1.923388e+28, p-value < 0.05) <br> `성별`의 주효과는 유의미하였으며1.516155e+28, p-value < 0.05)<br> `전공`과 `성별` 요인은 유의미한 상호작용효과를 가진다.(F(2,6) = 1.635269e+27, p-value < 0.05)

In [34]:
df6_1 = df6.query('전공 == 1')
model = ols('점수 ~ C(성별)', df6_1)
fit = model.fit()
anova_lm(fit)

,df,sum_sq,mean_sq,F,PR(>F)
C(성별),1.0,1.521000e-01,1.521000e-01,9.640483e+27,1.037292e-28
Residual,2.0,3.155444e-29,1.577722e-29,NaN,NaN


In [35]:
df6_2 = df6.query('전공 == 2')
model = ols('점수 ~ C(성별)', df6_2)
fit = model.fit()
anova_lm(fit)

,df,sum_sq,mean_sq,F,PR(>F)
C(성별),1.0,1.020100e+00,1.020100e+00,6.465652e+28,1.546634e-29
Residual,2.0,3.155444e-29,1.577722e-29,NaN,NaN


In [36]:
df6_3 = df6.query('전공 == 3')
model = ols('점수 ~ C(성별)', df6_3)
fit = model.fit()
anova_lm(fit)

,df,sum_sq,mean_sq,F,PR(>F)
C(성별),1.0,1.849000e-01,1.849000e-01,1.171943e+28,8.532838e-29
Residual,2.0,3.155444e-29,1.577722e-29,NaN,NaN


In [37]:
df6_4 = df6.query('성별 == 1')
model = ols('점수 ~ C(전공)', df6_4)
fit = model.fit()
anova_lm(fit)

,df,sum_sq,mean_sq,F,PR(>F)
C(전공),2.0,1.106533e+00,5.532667e-01,6.188372e+28,1.193362e-43
Residual,3.0,2.682127e-29,8.940424e-30,NaN,NaN


In [38]:
df6_5 = df6.query('성별 == 2')
model = ols('점수 ~ C(전공)', df6_5)
fit = model.fit()
anova_lm(fit)

,df,sum_sq,mean_sq,F,PR(>F)
C(전공),2.0,1.966533e+00,9.832667e-01,7.790241e+28,8.449102e-44
Residual,3.0,3.786532e-29,1.262177e-29,NaN,NaN


### 단순효과분석 결과

`전공`과 `성별`의 유의미한 상호작용 효과에 대하여 단순효과분석을 실시한 결과, `전공`이 1,2,3인 집단에서 각각 단순주효과는 유의미 하였다.<br>
`전공1` : (F(1,2) = 9.640483e+27 ,p < 0.05)<br>
`전공2` : (F(1,2)6.465652e+28 ,p < 0.05)<br>
`전공3` : (F(1,2)1.171943e+28 ,p < 0.05)<br>
`성별`이 1,2인 집단에서 각각 단순주효과는 유의미 하였다.<br>
`성별1` : (F(2,3) = 6.188372e+28, p < 0.05) <br>
`성별2` : (F(2,3) = 7.790241e+28, p < 0.05)

# 7)

In [39]:
df7 = DataFrame({'Kalium':[39.1,26.2,21.3,35.8,40.2,16.5,18.4,12.7,14.0,12.8,32.0,23.8,28.8,25.0,29.3,14.5,11.0,10.8,14.3,10.0],               
                'Hormone':['Yes','Yes','Yes','Yes','Yes','No','No','No','No','No','Yes','Yes','Yes','Yes','Yes','No','No','No','No','No'],                
                'Sex':['Female','Female','Female','Female','Female','Female','Female','Female','Female','Female','Male','Male','Male','Male','Male','Male','Male','Male','Male','Male']})
df7

,Kalium,Hormone,Sex
0,39.1,Yes,Female
1,26.2,Yes,Female
2,21.3,Yes,Female
3,35.8,Yes,Female
4,40.2,Yes,Female
5,16.5,No,Female
6,18.4,No,Female
7,12.7,No,Female
8,14.0,No,Female
9,12.8,No,Female


In [40]:
print(df7['Hormone'].unique())
print(df7['Sex'].unique())

['Yes' 'No']
['Female' 'Male']


In [41]:
df7['Hormone'] = df7['Hormone'].astype('category').cat.rename_categories({'Yes': 1, 'No': 2})
df7['Sex'] = df7['Sex'].astype('category').cat.rename_categories({'Male': 1, 'Female': 2})
unique = (df7['Hormone'].unique(), df7['Sex'].unique())
unique

([1, 2]
 Categories (2, int64): [1, 2],
 [2, 1]
 Categories (2, int64): [2, 1])

### 균형설계자료 확인

In [42]:
df7.groupby('Hormone').agg('count')

,Kalium,Sex
Hormone,,
2,10,10
1,10,10


- Hormone 요인으로 구분한 집단별 표본수는 모두 10으로 동일

In [43]:
df7.groupby('Sex').agg('count')

,Kalium,Hormone
Sex,,
2,10,10
1,10,10


- Sex 요인으로 구분한 집단별 표본수는 모두 10으로 동일 

In [44]:
df7.groupby(['Hormone', 'Sex']).agg('count')

Kalium
Hormone Sex        
2       2         5
        1         5
1       2         5
        1         5

- Hormone과 Sex로 구분된 집단별 표본수는 모두 5로 동일

- 모든 집단별 표본수가 동일하므로, 균형설계자료 -> 이원분산분석 가능

In [45]:
model = ols('Kalium ~ C(Hormone) * C(Sex)', df7)
fit = model.fit()
anova_lm(fit)

,df,sum_sq,mean_sq,F,PR(>F)
C(Hormone),1.0,1386.1125,1386.11250,60.533556,7.943078e-07
C(Sex),1.0,70.3125,70.31250,3.070650,9.885618e-02
C(Hormone):C(Sex),1.0,4.9005,4.90050,0.214012,6.498700e-01
Residual,16.0,366.3720,22.89825,NaN,NaN


### 결과분석
- Hormone: F(1,16) = 60.533556, p-value < 0.05로 유의미. 즉 Hormone 수준에 따라 평균에 차이가 난다고 볼 수 있다.
- Sex: F(1,16) = 3.070650, p-value > 0.05로 의미가 있다고 보기 힘들다. 즉 Sex는 평균과 관련이 없다고 볼 수 있다.
- Hormone, Sex: F(1,16) = 0.214012, p-value > 0.05로 의미가 있다고 보기 힘들다. 즉 Hormone, Sex는 평균과 관련이 없다고 볼 수 있다.

### 결과보고

`Kalium`에 대하여 `Hormone`과 `Sex`를 요인으로 하는 이원분산분석을 실시한 결과, `Hormone`의 주효과는 유의미하였으며(F(1,16) = 60.533556, p-value < 0.05) <br> `Sex`의 주효과는 유의미 하지 않았다.(F(1,16) = 3.070650, p-value > 0.05)<br> `Hormone`과 `Sex`요인의 유의미한 상호작용효과는 발견할 수 없었다.(F(1,16) = 0.214012, p-value > 0.05)